<a href="https://colab.research.google.com/github/anadiedrichs/2023-vision-computacional/blob/main/TP2_vision_Diedrichs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP2 - VISION COMPUTACIONAL

 Ana Laura Diedrichs
 SIU a0902
 9Co2022

## Ejercicio 1

Implementar la función ***create_gauss_filter(h, w, k_size, sigma)*** para crear filtros gaussianos para filtrado espectral. Debe retornar un filtro gaussiano de tamaño HxW en dominio espacial y su transformada de Fourier.

1.  Graficar ambas representaciones para diferentes tamaños de kernel y sigma. Aplicar el filtro una imagen para validar el funcionamiento en el dominio espectral.


In [ ]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
%matplotlib inline
#%matplotlib qt
# OpenCV-Python utiliza NumPy para el manejo de imágenes
import numpy as np
# cv2 es el módulo python para acceder a OpenCV
import cv2 as cv
# Usamos las poderosas herramientas de graficación de matplotlib para mostrar imágenes, perfiles, histogramas, etc
import matplotlib.pyplot as plt
# para importar imagenes desde urls
from skimage import io
# mostrar imagenes opencv en googlecolab
from google.colab.patches import cv2_imshow

In [ ]:
url = 'https://www.dropbox.com/s/8efkws4d0amkrgo/CoordCrom_1.png?dl=1'
#url = 'https://www.dropbox.com/s/8efkws4d0amkrgo/CoordCrom_2.png?dl=1'
#url = 'https://www.dropbox.com/s/8efkws4d0amkrgo/CoordCrom_3.png?dl=1'
myImg = io.imread(url)
image = cv.cvtColor(myImg, cv.COLOR_BGR2RGB)
#cv2_imshow(image) # descomentar para visualizar la imagen descargada

In [ ]:
#https://www.programcreek.com/python/example/89346/cv2.getGaussianKernel

In [ ]:
def create_gauss_filter(h, w, k_size, sigma):

    #TODO k_size debe ser impar

    #TODO h y w positivos

    # hacer padding de 0 hasta HxW
    img_1 = np.zeros([h,w,3],dtype=np.uint8)

    # Crear filtro gaussiano en el dominio espacial
    gauss_kernel = cv.getGaussianKernel(k_size, sigma)
    gauss_filter = np.outer(gauss_kernel, gauss_kernel.transpose())
    #print(gauss_filter)

    # viene normalizado el filtro ?
    # gauss_filter = gauss_filter / np.sum(gauss_filter)  # Normalizar el filtro ?

    nb = img_1.shape[0]
    na = gauss_filter.shape[0]
    lower = (nb) // 2 - (na // 2)
    upper = (nb // 2) + (na // 2)
    img_1[lower:upper+1, lower:upper+1,0] = gauss_filter
    img_1[lower:upper+1, lower:upper+1,1] = gauss_filter
    img_1[lower:upper+1, lower:upper+1,2] = gauss_filter

   # print(img_1[:, :,0])
   # result = cv.filter2D(img_1, -1 , gauss_filter)

    result = cv.GaussianBlur(img_1, ksize=(k_size, k_size), sigmaX=sigma)
    # Calcular la transformada de Fourier del filtro gaussiano
    # llevo la baja frec al origen para acomodar la visualizacion
    gauss_filter_fft = np.fft.fftshift(np.fft.fft2(result))
    # obtengo el modulo
    gauss_filter_fft = 20*np.log(np.abs(gauss_filter_fft))

    return result, gauss_filter_fft


In [ ]:
h = 256
w = 256
k_size = 11
sigma = 17.0

gauss_filter, gauss_filter_fft = create_gauss_filter(h, w, k_size, sigma)

cv2_imshow(gauss_filter) # "Gaussian Filter"


In [ ]:

cv2_imshow(gauss_filter_fft) # "Gaussian Filter FFT"

In [ ]:
import cv2
import numpy as np

def create_gauss_filter(h, w, k_size, sigma):
    # Crear una matriz vacía para el filtro gaussiano en el dominio espacial
    gauss_filter = np.zeros((h, w), dtype=np.float32)

    # Calcular el centro del filtro
    center_x = w // 2
    center_y = h // 2

    # Generar el filtro gaussiano en el dominio espacial
    for i in range(h):
        for j in range(w):
            gauss_filter[i, j] = np.exp(-((i - center_y) ** 2 + (j - center_x) ** 2) / (2 * sigma ** 2))

    # Normalizar el filtro
    gauss_filter /= np.sum(gauss_filter)

    # Calcular la transformada de Fourier del filtro
    gauss_filter_fft = np.fft.fftshift(np.fft.fft2(gauss_filter))

    return gauss_filter, gauss_filter_fft

# Ejemplo de uso
height = 256
width = 256
kernel_size = 31
sigma_value = 5

# Crear filtro gaussiano y su transformada de Fourier
gauss_filter, gauss_filter_fft = create_gauss_filter(height, width, kernel_size, sigma_value)

# Mostrar filtro gaussiano en el dominio espacial
cv2_imshow( gauss_filter) #"Gaussian Filter (Spatial Domain)",


# Mostrar filtro gaussiano en el dominio de la frecuencia (transformada de Fourier)
gauss_filter_fft_magnitude = 20 * np.log(np.abs(gauss_filter_fft))
gauss_filter_fft_magnitude = np.uint8(cv2.normalize(gauss_filter_fft_magnitude, None, 0, 255, cv2.NORM_MINMAX))
cv2_imshow( gauss_filter_fft_magnitude) #"Gaussian Filter (Frequency Domain)",



In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

def create_gauss_filter(h, w, k_size, sigma):

    # Crear un kernel gaussiano en el dominio espacial
    kernel = cv2.getGaussianKernel(k_size, sigma)
    # creo la matriz kernel de tamaño k_size x k_size
    kernel = np.outer(kernel, kernel)
    # Ampliar el kernel para que coincida con el tamaño deseado
    kernel = cv2.resize(kernel, (w, h))

    # Calcular la transformada de Fourier del kernel
    kernel_fft = np.fft.fft2(kernel)
    kernel_fft_shift = np.fft.fftshift(kernel_fft)

    return kernel, kernel_fft_shift


# Parámetros de prueba
height = 256
width = 256
kernel_size = 21
sigma_value = 5

# Crear filtro gaussiano y su transformada de Fourier
gauss_filter, gauss_filter_fft = create_gauss_filter(height, width, kernel_size, sigma_value)

# Visualizar el filtro gaussiano en el dominio espacial
plt.subplot(121), plt.imshow(gauss_filter, cmap='gray')
plt.title('Filtro Gaussiano (Dominio Espacial)'), plt.xticks([]), plt.yticks([])

plt.show()

# Visualizar la transformada de Fourier del filtro gaussiano
plt.subplot(122), plt.imshow(np.log(1 + np.abs(gauss_filter_fft)), cmap='gray')
plt.title('Transformada de Fourier del Filtro Gaussiano'), plt.xticks([]), plt.yticks([])

plt.show()


## Ejercicio 2

2. Usando el método descripto en el paper “Image Sharpness Measure for Blurred Images in Frequency Domain” comparar el resultado de un filtrado por convolución con el filtrado espectral.



## Ejercicio 3

3. Repetir la comparación usando uno de los métodos descriptos en el apéndice del paper “Analysis of focus measure operators in shape-from-focus”

Método A.24: gray-level variance (STA3)

$$ \phi_{x,y} = \sum_{i,j} (I(i,j)-μ)^2 $$



In [ ]:
# usando numpy as np
import numpy as np
from numpy.linalg import matrix_power

def gray_level_variance(image):
  media = np.mean(image)
  return matrix_power(image - media, 2)